In [0]:
cosmos_endpoint = "https://azurecosmosdb001.documents.azure.com:443/"
cosmos_key = "XjI6YzM5nnTsRp5kAu6od3o1BXk0yzDNpNKXE7rckKKLDlnVN8o0dAhRRddqFCq1MxNGpqOMHFmLACDbQCoKHQ=="
cosmos_database = "	CarsDB"
cosmos_container = "DataCoTable"

In [0]:
spark.conf.set("spark.azure.cosmosdb.accountEndpoint", cosmos_endpoint)
spark.conf.set("spark.azure.cosmosdb.accountKey", cosmos_key)
spark.conf.set("spark.azure.cosmosdb.database", cosmos_database)
spark.conf.set("spark.azure.cosmosdb.container", cosmos_container)

### 1.Read data from Cosmos DB

In [0]:
dataco = spark.read.format("cosmos.oltp") \
    .option("spark.cosmos.accountEndpoint", "https://azurecosmosdb001.documents.azure.com:443/") \
    .option("spark.cosmos.accountKey", "XjI6YzM5nnTsRp5kAu6od3o1BXk0yzDNpNKXE7rckKKLDlnVN8o0dAhRRddqFCq1MxNGpqOMHFmLACDbQCoKHQ==") \
    .option("spark.cosmos.database", "CarsDB") \
    .option("spark.cosmos.container", "DataCoTable") \
    .load()

dataco.show(5)

In [0]:
# Read data from Cosmos DB
logdata = spark.read.format("cosmos.oltp") \
    .option("spark.cosmos.accountEndpoint", "https://azurecosmosdb001.documents.azure.com:443/") \
    .option("spark.cosmos.accountKey", "XjI6YzM5nnTsRp5kAu6od3o1BXk0yzDNpNKXE7rckKKLDlnVN8o0dAhRRddqFCq1MxNGpqOMHFmLACDbQCoKHQ==") \
    .option("spark.cosmos.database", "CarsDB") \
    .option("spark.cosmos.container", "LogData") \
    .load()

logdata.show()

+----------+---------------+------------------+--------------------+-----+--------------------+----------------+--------------------+----+
|Department|             ip|          Category|                 url|Month|                  id|            Date|             Product|Hour|
+----------+---------------+------------------+--------------------+-----+--------------------+----------------+--------------------+----+
| outdoors | 203.10.210.179|  kids' golf clubs|/department/outdo...|  Dec|73ba8a7d-ae9c-438...|12/19/2017 10:12|Garmin Forerunner...|  10|
| outdoors | 51.219.249.191|women's golf clubs|/department/outdo...|  Dec|99eff4d1-e6f4-47f...| 12/19/2017 7:25|MDGolf Pittsburgh...|   7|
| outdoors |   3.230.167.77|       electronics|/department/outdo...|  Dec|fa04b9e2-26f0-475...| 12/19/2017 7:06|Bridgestone e6 St...|   7|
| outdoors |  68.26.199.113|women's golf clubs|/department/outdo...|  Nov|972c4cb2-3a5e-470...| 11/4/2017 21:19|Cleveland Golf Co...|  21|
| outdoors | 143.244.66.233

### 2.DataCo Data Transformation

#### Shape of the Data

In [0]:
dataco.cache()

Out[6]: DataFrame[Product Price: string, Order Id: string, Customer Zipcode: string, Product Description: void, Shipping Mode: string, Category Id: string, Department Name: string, Order Item Product Price: string, Sales: string, Product Card Id: string, Latitude: string, Order Item Discount: string, Customer Lname: string, Order Country: string, Customer Fname: string, Longitude: string, order date (DateOrders): string, Product Status: string, Order Item Quantity: string, Customer Segment: string, Customer Country: string, Product Name: string, Order Item Discount Rate: string, Order Item Profit Ratio: string, Customer Id: string, Order Zipcode: string, Customer State: string, Product Image: string, Order State: string, Benefit per order: string, Category Name: string, Market: string, Customer Password: string, Order Profit Per Order: string, id: string, Department Id: string, Customer Email: string, Order Item Id: string, Delivery Status: string, Days for shipment (scheduled): string

In [0]:
dataco.count()

Out[7]: 180519

In [0]:
len(dataco.columns)

Out[8]: 54

In [0]:
from pyspark.sql.functions import col, sum

null_counts = dataco.select([sum(col(c).isNull().cast("int")).alias(c) for c in dataco.columns])

# Display the null counts for each column
null_counts.show()


+-------------+--------+----------------+-------------------+-------------+-----------+---------------+------------------------+-----+---------------+--------+-------------------+--------------+-------------+--------------+---------+-----------------------+--------------+-------------------+----------------+----------------+------------+------------------------+-----------------------+-----------+-------------+--------------+-------------+-----------+-----------------+-------------+------+-----------------+----------------------+---+-------------+--------------+-------------+---------------+-----------------------------+-------------+------------+------------------------+-----------------+-------------------+------------+---------------+----------------------+------------------+----------------+------------------+----------+----+--------------------------+
|Product Price|Order Id|Customer Zipcode|Product Description|Shipping Mode|Category Id|Department Name|Order Item Product Price|Sal

#### Remove null values

In [0]:
from pyspark.sql.functions import col, sum

null_counts = dataco.select([sum(col(c).isNull().cast("int")).alias(c) for c in cleaned_df1.columns])

null_counts.show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3132932548810543>:3
      1 from pyspark.sql.functions import col, sum
----> 3 null_counts = dataco.select([sum(col(c).isNull().cast("int")).alias(c) for c in cleaned_df1.columns])
      5 null_counts.show()

NameError: name 'cleaned_df1' is not defined

In [0]:
# Collecting the null counts as a list
null_counts_list = null_counts.collect()[0]

# Creating a dictionary for key-value pairs
null_counts_dict = {null_counts.columns[i]: null_counts_list[i] for i in range(len(null_counts.columns))}

# Display key-value pairs
for column, count in null_counts_dict.items():
    print(f"{column}: {count}")


Product Price: 0
Order Id: 0
Customer Zipcode: 3
Product Description: 180519
Shipping Mode: 0
Category Id: 0
Department Name: 0
Order Item Product Price: 0
Sales: 0
Product Card Id: 0
Latitude: 0
Order Item Discount: 0
Customer Lname: 8
Order Country: 0
Customer Fname: 0
Longitude: 0
order date (DateOrders): 0
Product Status: 0
Order Item Quantity: 0
Customer Segment: 0
Customer Country: 0
Product Name: 0
Order Item Discount Rate: 0
Order Item Profit Ratio: 0
Customer Id: 0
Order Zipcode: 155679
Customer State: 0
Product Image: 0
Order State: 0
Benefit per order: 0
Category Name: 0
Market: 0
Customer Password: 0
Order Profit Per Order: 0
id: 0
Department Id: 0
Customer Email: 0
Order Item Id: 0
Delivery Status: 0
Days for shipment (scheduled): 0
Customer City: 0
Order Status: 0
Days for shipping (real): 0
Order Customer Id: 0
Product Category Id: 0
Order Region: 0
Customer Street: 0
Order Item Cardprod Id: 0
Late_delivery_risk: 0
Order Item Total: 0
Sales per customer: 0
Order City: 0


#### Renaming the Columns using _

In [0]:
def rename_col(s):
    s = s.replace(' ', '_')
    s = s.replace('-', '_')
    s = s.replace('(', '')
    s = s.replace(')', '')
    a = s.split('_')
    for i in range(len(a)):
        if not all([c == c.upper() for c in a[i]]):
            a[i] = a[i].capitalize()
    return '_'.join(a)

# Apply the function to all column names in PySpark DataFrame
for column in dataco.columns:
    new_column = rename_col(column)
    dataco = dataco.withColumnRenamed(column, new_column)

# Show the DataFrame with renamed columns
dataco.printSchema()  # To show the updated column names


root
 |-- Product_Price: string (nullable = true)
 |-- Order_Id: string (nullable = true)
 |-- Customer_Zipcode: string (nullable = true)
 |-- Product_Description: void (nullable = true)
 |-- Shipping_Mode: string (nullable = true)
 |-- Category_Id: string (nullable = true)
 |-- Department_Name: string (nullable = true)
 |-- Order_Item_Product_Price: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Product_Card_Id: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Order_Item_Discount: string (nullable = true)
 |-- Customer_Lname: string (nullable = true)
 |-- Order_Country: string (nullable = true)
 |-- Customer_Fname: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Order_Date_Dateorders: string (nullable = true)
 |-- Product_Status: string (nullable = true)
 |-- Order_Item_Quantity: string (nullable = true)
 |-- Customer_Segment: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Product_Name: stri

In [0]:
#Filling Null values
dataco = dataco.fillna({'Customer_Zipcode':0,'Customer_Lname':'NA'})

#### Drop Columns

In [0]:
#Removing Columns which are not required
cleaned_df = dataco.drop('Order_Zipcode','Customer_Email','Customer_Password','Product_Description','id','Product_Image')

In [0]:
#Droping the duplicate rows
cleaned_df = cleaned_df.dropDuplicates()

In [0]:
cleaned_df.count()

Out[16]: 180519

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


In [0]:

from pyspark.sql.functions import col, sum

null_counts = cleaned_df1.select([sum(col(c).isNull().cast("int")).alias(c) for c in cleaned_df1.columns])

null_counts.show()


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1556790937502333>:3
      1 from pyspark.sql.functions import col, sum
----> 3 null_counts = cleaned_df1.select([sum(col(c).isNull().cast("int")).alias(c) for c in cleaned_df1.columns])
      5 null_counts.show()

NameError: name 'cleaned_df1' is not defined

#### TypeCast Column values

In [0]:
cleaned_df1.printSchema()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3135515182037910>:1
----> 1 cleaned_df1.printSchema()

NameError: name 'cleaned_df1' is not defined

In [0]:
from pyspark.sql.types import IntegerType, FloatType, DateType,TimestampType
from pyspark.sql.functions import *

In [0]:


# Casting the integer, float, and date columns

cleaned_df1 = cleaned_df.withColumn("Category_Id", cleaned_df["Category_Id"].cast(IntegerType())) \
    .withColumn("Customer_Id", cleaned_df["Customer_Id"].cast(IntegerType())) \
    .withColumn("Customer_Zipcode", cleaned_df["Customer_Zipcode"].cast(IntegerType())) \
    .withColumn("Department_Id", cleaned_df["Department_Id"].cast(IntegerType())) \
    .withColumn("Latitude", cleaned_df["Latitude"].cast(FloatType())) \
    .withColumn("Longitude", cleaned_df["Longitude"].cast(FloatType())) \
    .withColumn("Order_Customer_Id", cleaned_df["Order_Customer_Id"].cast(IntegerType())) \
    .withColumn("Order_Id", cleaned_df["Order_Id"].cast(IntegerType())) \
    .withColumn("Order_Item_Cardprod_Id", cleaned_df["Order_Item_Cardprod_Id"].cast(IntegerType())) \
    .withColumn("Order_Item_Discount", cleaned_df["Order_Item_Discount"].cast(FloatType())) \
    .withColumn("Order_Item_Discount_Rate", cleaned_df["Order_Item_Discount_Rate"].cast(FloatType())) \
    .withColumn("Order_Item_Id", cleaned_df["Order_Item_Id"].cast(IntegerType())) \
    .withColumn("Order_Item_Product_Price", cleaned_df["Order_Item_Product_Price"].cast(FloatType())) \
    .withColumn("Order_Item_Profit_Ratio", cleaned_df["Order_Item_Profit_Ratio"].cast(FloatType())) \
    .withColumn("Order_Item_Quantity", cleaned_df["Order_Item_Quantity"].cast(IntegerType())) \
    .withColumn("Sales", cleaned_df["Sales"].cast(FloatType())) \
    .withColumn("Order_Item_Total", cleaned_df["Order_Item_Total"].cast(FloatType())) \
    .withColumn("Order_Profit_Per_Order", cleaned_df["Order_Profit_Per_Order"].cast(FloatType())) \
    .withColumn("Product_Card_Id", cleaned_df["Product_Card_Id"].cast(IntegerType())) \
    .withColumn("Product_Category_Id", cleaned_df["Product_Category_Id"].cast(IntegerType())) \
    .withColumn("Product_Price", cleaned_df["Product_Price"].cast(FloatType())) \
    .withColumn("Days_For_Shipping_Real", cleaned_df["Days_For_Shipping_Real"].cast(IntegerType())) \
    .withColumn("Days_For_Shipment_Scheduled", cleaned_df["Days_For_Shipment_Scheduled"].cast(IntegerType())) \
    .withColumn("Benefit_Per_Order", cleaned_df["Benefit_Per_Order"].cast(FloatType())) \
    .withColumn("Sales_Per_Customer", cleaned_df["Sales_Per_Customer"].cast(FloatType())) \
    .withColumn("Late_delivery_risk", cleaned_df["Late_delivery_risk"].cast(IntegerType()))


In [0]:
cleaned_df1 = cleaned_df1.withColumn("Order_Date_Dateorders",when(col("Order_Date_Dateorders").contains('-'),to_timestamp(col("Order_Date_Dateorders"), "MM-dd-yyyHH:mm")).otherwise(to_timestamp(col("Order_Date_Dateorders"), "MM/dd/yyyy H:mm"))) \
            .withColumn("Shipping_Date_Dateorders",when(col("Shipping_Date_Dateorders").contains('-'),to_timestamp(col("Shipping_Date_Dateorders"), "MM-dd-yyyHH:mm")).otherwise(to_timestamp(col("Shipping_Date_Dateorders"), "MM/dd/yyyy H:mm")))

In [0]:
from pyspark.sql.types import StringType, NumericType

schema = cleaned_df1.dtypes

# Separate categorical and numerical columns
categorical_columns = [name for name, dtype in schema if isinstance(cleaned_df1.schema[name].dataType, StringType)]
numerical_columns = [name for name, dtype in schema if isinstance(cleaned_df1.schema[name].dataType, NumericType)]

# Display the lists
print("Categorical Columns:")
print(categorical_columns)

print("Numerical Columns:")
print(numerical_columns)


Categorical Columns:
['Shipping_Mode', 'Department_Name', 'Customer_Lname', 'Order_Country', 'Customer_Fname', 'Product_Status', 'Customer_Segment', 'Customer_Country', 'Product_Name', 'Customer_State', 'Order_State', 'Category_Name', 'Market', 'Delivery_Status', 'Customer_City', 'Order_Status', 'Order_Region', 'Customer_Street', 'Order_City', 'Type']
Numerical Columns:
['Product_Price', 'Order_Id', 'Customer_Zipcode', 'Category_Id', 'Order_Item_Product_Price', 'Sales', 'Product_Card_Id', 'Latitude', 'Order_Item_Discount', 'Longitude', 'Order_Item_Quantity', 'Order_Item_Discount_Rate', 'Order_Item_Profit_Ratio', 'Customer_Id', 'Benefit_Per_Order', 'Order_Profit_Per_Order', 'Department_Id', 'Order_Item_Id', 'Days_For_Shipment_Scheduled', 'Days_For_Shipping_Real', 'Order_Customer_Id', 'Product_Category_Id', 'Order_Item_Cardprod_Id', 'Late_delivery_risk', 'Order_Item_Total', 'Sales_Per_Customer']


In [0]:
cleaned_df1.show(5)

+-------------+--------+----------------+--------------+-----------+---------------+------------------------+------+---------------+---------+-------------------+--------------+-------------+--------------+----------+---------------------+--------------+-------------------+----------------+----------------+--------------------+------------------------+-----------------------+-----------+--------------+--------------------+-----------------+-----------------+------+----------------------+-------------+-------------+----------------+---------------------------+--------------+---------------+----------------------+-----------------+-------------------+---------------+--------------------+----------------------+------------------+----------------+------------------+----------+-------+------------------------+
|Product_Price|Order_Id|Customer_Zipcode| Shipping_Mode|Category_Id|Department_Name|Order_Item_Product_Price| Sales|Product_Card_Id| Latitude|Order_Item_Discount|Customer_Lname|Order_

#### Shape after Transformations

In [0]:
cleaned_df1.count()

Out[25]: 180519

In [0]:
len(cleaned_df1.columns)

Out[26]: 48

#### Store the Transformed data into Blob Storage(Parquet Format)

In [0]:
storage_account_name = "capstonestorage001"
storage_account_key = "BY59d83vF22ri+DpwUKIc4Tgbv5oukkSw+KL/YnW8I7w2d1+xqXp/LTInlMLht8KeGFHKrj/j4/o+AStb5Bs0A=="
container_name = "datacotransformed"
 
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)
 
output_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/transformed_data/"
   
cleaned_df1.write.mode("overwrite").option("header", "true").parquet(output_path)

### 3.LogData Transformation

In [0]:
logdata.cache()

Out[28]: DataFrame[Department: string, ip: string, Category: string, url: string, Month: string, id: string, Date: string, Product: string, Hour: string]

In [0]:
logdata = logdata.dropDuplicates()

In [0]:
logdata = logdata.drop('id')

In [0]:
logdata.count()

Out[31]: 469977

In [0]:
logdata = logdata.withColumn("Date", when(col("Date").contains('-'),to_timestamp(col("Date"), "MM-dd-yyy HH:mm"))\
          .otherwise(to_timestamp(col("Date"), "MM/dd/yyyy H:mm"))) \
          .withColumn("Hour",logdata["Hour"].cast(IntegerType()))

In [0]:
logdata.printSchema()

root
 |-- Department: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- url: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Product: string (nullable = true)
 |-- Hour: integer (nullable = true)



In [0]:
storage_account_name = "capstonestorage001"
storage_account_key = "BY59d83vF22ri+DpwUKIc4Tgbv5oukkSw+KL/YnW8I7w2d1+xqXp/LTInlMLht8KeGFHKrj/j4/o+AStb5Bs0A=="
container_name = "transformation"
 
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)
 
output_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/transformed_data/"
   
logdata.write.mode("overwrite").option("header", "true").parquet(output_path)